<a href="https://colab.research.google.com/github/visiont3lab/deep-learning-course/blob/main/colab/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library

In [1]:
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
import torch
from torch import nn
from torchsummary import summary
#!pip install torchsummary
import torch.nn.functional as F
from torch.utils.data import TensorDataset,Dataset
from torchvision import datasets
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import plotly.graph_objects as go
# Loss function pytorch: https://neptune.ai/blog/pytorch-loss-functions
import copy
import pandas as pd
from datetime import datetime

## Neural Network 

## Dataset

In [2]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/visiont3lab/deep-learning-course/main/data/covid19-ita-regioni.csv")
df["data"] = [ datetime.strptime(d, "%Y-%m-%d") for d in df["data"]]
#df_f = df[df["data"]>datetime(2020,11,6)].copy()
df_f = df[df["zona"]!="unknown"]
inputs = ["data","denominazione_regione","zona","ricoverati_con_sintomi","terapia_intensiva",
        "totale_ospedalizzati","totale_positivi","isolamento_domiciliare",
        "deceduti","dimessi_guariti","nuovi_positivi","totale_casi","tamponi"]
df_f = df_f[inputs]
df_f = df_f.drop(columns=["data","denominazione_regione"])
#display(df_f)

#Y = np.array([dict_names[d] for d in df_Y],dtype=np.float) #.reshape(-1,1)
#print(f"X shape: {X.shape} , Y shape: {Y.shape}")

dict_names = {"bianca":0,"gialla": 1, "arancione": 2, "rossa": 3}
Y = df_f.pop("zona").tolist()
Y = np.array ( [ dict_names[e] for e in Y] , dtype=np.float32).reshape(-1,1)

X = df_f.values

print( X.shape )
print( Y.shape )

(2689, 10)
(2689, 1)


## Neural Network


In [3]:
class ClassificationNet(nn.Module):
    def __init__(self,num_inputs, num_classes):
        super(ClassificationNet,self).__init__()
        self.num_classes = num_classes
        self.fc1 = nn.Linear(num_inputs,200)
        self.fc2 = nn.Linear(200,100)
        self.fc3 = nn.Linear(100,50)
        self.fc4 = nn.Linear(50,self.num_classes)
    def forward(self,x):
        # torch.sigmoid, torch.tanh, torch.relu
        x = torch.tanh(self.fc1(x)) 
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.log_softmax(self.fc4(x),dim=-1) # sarebbe dim=1  print(self.fc3(x)) print(self.fc3(x).sum(dim=-1))
        return x

CN = ClassificationNet(num_inputs=10, num_classes=4)
summary(CN,input_size=(1,10),batch_size=-1, device='cpu')
#print(CN)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 200]           2,200
            Linear-2               [-1, 1, 100]          20,100
            Linear-3                [-1, 1, 50]           5,050
            Linear-4                 [-1, 1, 4]             204
Total params: 27,554
Trainable params: 27,554
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.11
Estimated Total Size (MB): 0.11
----------------------------------------------------------------


## Preprocessing

In [4]:
# Training and Test Set
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,shuffle=True,random_state=2)
print(f"X Train shape: {X_train.shape} , X Test shape: {X_test.shape}")

# Normalization
C_mean = np.mean(X)
C_std = np.std(X)
C_min = np.min(X)
C_max = np.max(X)

# Tensor Dataset Che converte i dati da numpy a Pytorch
class CustomTensorDataset(Dataset):
    def __init__(self, x,y,mean,std):
        x = (x - mean)/std           # Standard Scaler 
        #x = (x - min) / (max - min) # Min Max Scaler
        self.x = torch.from_numpy(x).type(torch.float32)
        self.y = torch.from_numpy(y).type(torch.LongTensor).reshape(-1)
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y
    def __len__(self):
        return self.x.shape[0]

# Dataset generator creation
train_ds = CustomTensorDataset(X_train,Y_train,C_mean,C_std)
test_ds = CustomTensorDataset(X_test,Y_test,C_mean,C_std)

X Train shape: (1882, 10) , X Test shape: (807, 10)


## Training

* numero di epoche
* learning rate
* batch size
* aggiornare la struttura della rete

In [5]:
# validation: metric regression
def metrics_func_regression(target, output):
    # Comptue mean squaer error (Migliora quanto piu' ci avviciniamo a zero)
    mse = torch.sum((output - target) ** 2)
    return mse

# validation metric classification
def metrics_func_classification(target, output):
    # Compute number of correct prediction
    pred = output.argmax(dim=-1,keepdim=True)
    corrects =pred.eq(target.reshape(pred.shape)).sum().item()
    return -corrects # minus for coeherence with best result is the most negative one

# training: loss calculation and backward step
def loss_batch(loss_func,metric_func, xb,yb,yb_h, opt=None):
    # obtain loss
    loss = loss_func(yb_h, yb)
    # obtain performance metric 
    with torch.no_grad():
        metric_b = metric_func(yb,yb_h)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.item(), metric_b

# one epoch training
def loss_epoch(model, loss_func,metric_func, dataset_dl, sanity_check,opt, device):
    loss = 0.0
    metric = 0.0
    len_data = float(len(dataset_dl.dataset))
    # get batch data
    for xb,yb in dataset_dl:    
        # send to cuda the data (batch size)
        xb = xb.to(device)
        yb = yb.to(device)
        # obtain model output 
        yb_h = model.forward(xb)
        # loss and metric Calculation
        loss_b, metric_b = loss_batch(loss_func,metric_func, xb,yb,yb_h,opt)
        # update loss
        loss += loss_b
        # update metric
        if metric_b is not None:
            metric+=metric_b 
        if sanity_check is True:
            break
    # average loss
    loss /=len_data
    # average metric
    metric /=len_data
    return loss, metric

# get learning rate from optimizer
def get_lr(opt):
    # opt.param_groups[0]['lr']
    for param_group in opt.param_groups:
        return param_group["lr"]

# trainig - test loop
def train_test(params):
    # --> extract params
    model = params["model"]
    loss_func=params["loss_func"]
    metric_func=params["metric_func"]
    num_epochs=params["num_epochs"]
    opt=params["optimizer"]
    lr_scheduler=params["lr_scheduler"]
    train_dl=params["train_dl"]
    test_dl=params["test_dl"]
    device=params["device"]
    continue_training=params["continue_training"]
    sanity_check=params["sanity_check"]
    path2weigths=params["path2weigths"]
    # --> send model to device and print device
    model = model.to(device)
    print("--> training device %s" % (device))
    # --> if continue_training=True load path2weigths
    if continue_training==True and os.path.isfile(path2weigths):
        print("--> continue training  from last best weights")
        weights = torch.load(path2weigths)
        model.load_state_dict(weights)
    # --> history of loss values in each epoch
    loss_history={"train": [],"test":[]}
    # --> history of metric values in each epoch
    metric_history={"train": [],"test":[]}
    # --> a deep copy of weights for the best performing model
    best_model_weights = copy.deepcopy(model.state_dict())
    # --> initialiaze best loss to large value
    best_loss=float("inf")
    # --> main loop
    for epoch in range(num_epochs):
        # --> get learning rate
        lr = get_lr(opt)
        print("----\nEpoch %s/%s, lr=%.6f" % (epoch+1,num_epochs,lr))
        # --> train model on training dataset
        # we tell to the model to enter in train state. it is important because
        # there are somelayers like dropout, batchnorm that behaves 
        # differently between train and test
        model.train()
        train_loss,train_metric = loss_epoch(model, loss_func, metric_func,train_dl,sanity_check, opt,device)
        # --> collect loss and metric for training dataset
        loss_history["train"].append(train_loss)
        metric_history["train"].append(train_metric)
        # --> tell the model to be in test (validation) mode
        model.eval()
        with torch.no_grad():
            test_loss, test_metric = loss_epoch(model, loss_func, metric_func, test_dl,sanity_check,opt=None,device=device)
        # --> collect loss and metric for test dataset
        loss_history["test"].append(test_loss)
        metric_history["test"].append(test_metric)
        # --> store best model
        if test_loss < best_loss:
            print("--> model improved! --> saved to %s" %(path2weigths))
            best_loss = test_loss
            best_model_weights = copy.deepcopy(model.state_dict())
            # --> store weights into local file
            torch.save(model.state_dict(),path2weigths)
        # --> learning rate scheduler
        lr_scheduler.step()
        print("--> train_loss: %.6f, test_loss: %.6f, train_metric: %.3f, test_metric: %.3f" % (train_loss,test_loss,train_metric,test_metric))
    # --> load best weights
    model.load_state_dict(best_model_weights)
    return model, loss_history,metric_history


In [7]:
# Setup GPU Device
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")

# Regression
model = ClassificationNet(num_inputs=10, num_classes=4).to(device)
loss_func = nn.NLLLoss(reduction="sum")  #nn.BCELoss  
opt = optim.Adam(model.parameters(),lr=0.001)
train_dl = DataLoader(train_ds,batch_size=124,shuffle=True)
test_dl = DataLoader(test_ds,batch_size=124,shuffle=True)

# Regression

# Setup GPU Device
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.999)  #  lr = lr * gamma ** last_epoch
params = {
    "model":                 model,
    "loss_func":             loss_func, 
    "metric_func":           metrics_func_classification,
    "num_epochs":            300,
    "optimizer":             opt,
    "lr_scheduler":          lr_scheduler,
    "train_dl":              train_dl,
    "test_dl":               test_dl,
    "device":                device,  
    "continue_training" :    False,  # continue training from last save weights
    "sanity_check":          False, # if true we only do one batch per epoch
    "path2weigths":          "./weights_classification.pt"  
} 
model, loss_history,metric_history = train_test(params)

--> training device cpu
----
Epoch 1/300, lr=0.001000
--> model improved! --> saved to ./weights_classification.pt
--> train_loss: 1.170238, test_loss: 1.102329, train_metric: -0.370, test_metric: -0.372
----
Epoch 2/300, lr=0.000999
--> model improved! --> saved to ./weights_classification.pt
--> train_loss: 1.105318, test_loss: 1.095627, train_metric: -0.386, test_metric: -0.413
----
Epoch 3/300, lr=0.000998
--> train_loss: 1.101353, test_loss: 1.096018, train_metric: -0.397, test_metric: -0.413
----
Epoch 4/300, lr=0.000997
--> model improved! --> saved to ./weights_classification.pt
--> train_loss: 1.106576, test_loss: 1.094528, train_metric: -0.389, test_metric: -0.372
----
Epoch 5/300, lr=0.000996
--> train_loss: 1.102463, test_loss: 1.095452, train_metric: -0.410, test_metric: -0.456
----
Epoch 6/300, lr=0.000995
--> train_loss: 1.100696, test_loss: 1.097028, train_metric: -0.419, test_metric: -0.372
----
Epoch 7/300, lr=0.000994
--> model improved! --> saved to ./weights_classi

## Testing

In [10]:
# Load Classification
names = list({"bianca":0,"gialla": 1, "arancione": 2, "rossa": 3})
model = ClassificationNet(num_inputs=10,num_classes=4).to(device)
weights = torch.load("weights_classification.pt")
model.load_state_dict(weights)
model = model.to(device)

# Predict Classication
# C_mean, C_std = 186785.728337672 679353.7516865473
Xt = torch.from_numpy(X_test).type(torch.float32).unsqueeze(1)
Xt = (Xt - C_mean) / C_std
Y_hat = model.forward(Xt).argmax(dim=-1,keepdim=True).detach().numpy().reshape(-1)

# Visualize results
cm = confusion_matrix(Y_test,Y_hat)
names_pred = [ "Pred: " + n for n in names]
df = pd.DataFrame(cm, columns=names_pred, index=names)
print(df)

           Pred: bianca  Pred: gialla  Pred: arancione  Pred: rossa
bianca                3             0                3            0
gialla                8           212               53           27
arancione             0            60              240           33
rossa                 0            51               37           80


## Extra

In [ ]:

# df.head() # prime 5 righe
# df.tail() # ultime 5 righe
# df.to_csv("a.csv") # Salva come csv
# df.keys() # nomi delle colonne
# df.values.shape # dimensione
# df.values # dati come array 

# Nomi delle colonne
# Numero di righe
# selezionare solo la colonna con nome dimessi_guariti
# voglio i dati come array
# voglio i deceduti > 700
# dati del 12 marzo in poi

In [ ]:
x = np.array( [5,3,5,6,7] , dtype=np.float32)
x = x.reshape(1,-1)
x

array([[5., 3., 5., 6., 7.]], dtype=float32)

In [ ]:
x = torch.tensor( [5,3,5,6,7] , dtype=torch.float32)
#x = x.reshape(1,-1)
#x = x.unsqueeze(0)
x.shape

torch.Size([5, 1])

In [ ]:
import pandas as pd

d = {
    "c" : [5,6,7,8,9],
    "e" : [15,26,37,18,29],
    "t" : [0.5,0.26,3.7,1.8,29], 
}

df = pd.DataFrame(d)
df.values

array([[ 5.  , 15.  ,  0.5 ],
       [ 6.  , 26.  ,  0.26],
       [ 7.  , 37.  ,  3.7 ],
       [ 8.  , 18.  ,  1.8 ],
       [ 9.  , 29.  , 29.  ]])

In [ ]:
x = torch.from_numpy(df.values).type(torch.float32)
x

tensor([[ 5.0000, 15.0000,  0.5000],
        [ 6.0000, 26.0000,  0.2600],
        [ 7.0000, 37.0000,  3.7000],
        [ 8.0000, 18.0000,  1.8000],
        [ 9.0000, 29.0000, 29.0000]])